In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime 
import locale
locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')

import warnings
warnings.filterwarnings(action='ignore')

import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')

hj = pd.read_excel('급속충전이용내역_환경부_18.01-19.10.xlsx', encoding = "CP949")
hg = pd.read_excel('급속충전이용내역_한전_18.01-19.10.xlsx', encoding = "CP949")

hj2 = hj.copy()
hg2 = hg.copy()

hj2.rename(columns = {
    '충전소명' : '충전소',
    '충전소대분류' : '용도',
    '충전시작시간' : '충전날짜',
    '충전량(kWh)' : '충전량_kWh'
}, inplace = True)

hg2.rename(columns = {
    '용도별' : '용도',
    '충전소 구축일' : '설치년도',
    '시작일시' : '충전날짜',
    '충전량(kWh)' : '충전량_kWh'
}, inplace = True)

hj2 = hj2.loc[:, ['충전소', '주소', '용도','설치년도', '충전량_kWh', '충전날짜']]
hg2 = hg2.loc[:, ['충전소', '주소', '용도', '설치년도','충전량_kWh', '충전날짜']]

hg2['설치년도'] = list(map(lambda x : x.split('-')[0], hg2['설치년도']))

hj2['충전날짜'] = list(map(lambda x : str(x), hj2['충전날짜'] ))
hj2['충전날짜'] = list(map(lambda x : x[2:10], hj2['충전날짜'] ))

hg2['충전날짜'] = list(map(lambda x : x.split(' ')[0].replace('-', '')[2:] + x.split(' ')[1].replace(':', '')[:2], hg2['충전날짜'] ))

hj2['연도'] = list(map(lambda x : x[:2], hj2['충전날짜']))
hj2['연월'] = list(map(lambda x : x[:4], hj2['충전날짜']))
hj2['연월일'] = list(map(lambda x : x[:6], hj2['충전날짜']))
hj2['월'] = list(map(lambda x : x[2:4], hj2['충전날짜']))
hj2['시간'] = list(map(lambda x : x[6:], hj2['충전날짜']))
hj2['시간'] = list(map(lambda x : int(x) , hj2['시간']))

hg2['연도'] = list(map(lambda x : x[:2], hg2['충전날짜']))
hg2['연월'] = list(map(lambda x : x[:4], hg2['충전날짜']))
hg2['연월일'] = list(map(lambda x : x[:6], hg2['충전날짜']))
hg2['월'] = list(map(lambda x : x[2:4], hg2['충전날짜']))
hg2['시간'] = list(map(lambda x : x[6:], hg2['충전날짜']))
hg2['시간'] = list(map(lambda x : int(x) , hg2['시간']))

hj3 = hj2.copy()

hj4 = hj3.drop(hj3[hj3['연월'] == '1910'].index, 0)
hj4.index = range(len(hj4))
hj4.shape

use_ch = pd.concat([hg2, hj4], axis = 0, ignore_index = True)
print(use_ch.shape)
use_ch.head(5)

use_ch2 = use_ch.copy()
use_ch2['시간'] = list(map(lambda x : str(x), use_ch2['시간']))
#ch_ym = use_ch2.groupby(['충전소', '연월']).sum()

for i in use_ch2['충전소'].unique()[0] :
    ch_ym = use_ch2[use_ch2['충전소'] == i].groupby(['연월']).sum()
    ch_ym['연월'] = ch_ym.index
    ch_ym.index = range(1, len(ch_ym)+1)
    ch_ym['순서'] = ch_ym.index


'''import statsmodels.api as sm
import statsmodels.formula.api as smf

beta1_li = []

for i in use_ch2['충전소'].unique() :
    ch_ym = use_ch2[use_ch2['충전소'] == i].groupby(['연월']).sum()
    ch_ym['연월'] = ch_ym.index
    ch_ym.index = range(1, len(ch_ym)+1)
    ch_ym['순서'] = ch_ym.index
    
    # 회귀식 
    model = smf.ols(formula = '충전량_kWh ~ 순서', data = ch_ym)
    result = model.fit()
    result.summary()'''

from sklearn.linear_model import LinearRegression

ch_name = []
beta1_li = []

for i in use_ch2['충전소'].unique() :
    ch_ym = use_ch2[use_ch2['충전소'] == i].groupby(['연월']).sum()
    ch_ym['연월'] = ch_ym.index
    ch_ym.index = range(1, len(ch_ym)+1)
    ch_ym['순서'] = ch_ym.index
    
    ch_name.append(i)
    #print(ch_ym)
    model = LinearRegression().fit(pd.DataFrame(ch_ym['충전량_kWh']), pd.DataFrame(ch_ym['순서']))
    beta1_li.append(float(model.coef_))
    
result = pd.DataFrame( [ch_name, beta1_li], index = ['충전소', 'Beta1']).T
result

result['번호'] = result.index
result

result

(89531, 11)


,충전소,Beta1,번호
0,강화지사,0.00637067,0
1,강화군청,0.00710396,1
2,신문리 공영주차장,0.00358393,2
3,온수리 제1공영주차장,0.0141114,3
4,마니산국민관광지,0.0105803,4
5,초정마을동아 아파트,0.0429987,5
6,계양구청,0.00441788,6
7,홈플러스 작전점,0.00753529,7
8,하나로마트 인천점,0.00489308,8
9,홈플러스 계산점,0.00905446,9


In [3]:
res_info = pd.read_excel("12.30 충전소명 및 군구.xlsx", encoding = "CP949")

In [4]:
res_info

,충전소,시군구명
0,G-Tower(경제자유구역청),연수구
1,LH인천본부,남동구
2,lh웨스턴블루힐,서구
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,서구
4,가정뉴서울아파트,서구
5,가좌2동 행정복지센터,서구
6,간석래미안자이,남동구
7,갈산역 공영주차장,부평구
8,갈산이안 아파트,부평구
9,강화군청,강화군


In [5]:
T = pd.merge(result, res_info, on = '충전소', how = 'left')

In [6]:
T

,충전소,Beta1,번호,시군구명
0,강화지사,0.00637067,0,강화군
1,강화군청,0.00710396,1,강화군
2,신문리 공영주차장,0.00358393,2,강화군
3,온수리 제1공영주차장,0.0141114,3,강화군
4,마니산국민관광지,0.0105803,4,강화군
5,초정마을동아 아파트,0.0429987,5,계양구
6,계양구청,0.00441788,6,계양구
7,홈플러스 작전점,0.00753529,7,계양구
8,하나로마트 인천점,0.00489308,8,계양구
9,홈플러스 계산점,0.00905446,9,계양구


In [9]:
T2 = T.sort_values(['Beta1'], ascending = False)
T2.index = range(len(T2))
T2

,충전소,Beta1,번호,시군구명
0,검단아이파크아파트,0.0745724,59,서구
1,삼산서해그랑블 아파트,0.0685162,45,부평구
2,송도더샵그린워크1차,0.06297,86,연수구
3,작전동보2차 아파트,0.0597745,10,계양구
4,영종동보노빌리티아파트,0.0531233,107,중구
5,청라더샵레이크파크,0.0496901,72,서구
6,송도푸르지오하버뷰,0.0490451,94,연수구
7,용현엑슬루타워 아파트,0.0488442,34,미추홀구
8,중부발전사원 아파트,0.0478907,50,서구
9,청라웰카운티19단지,0.0463229,64,서구


In [10]:
T2.to_csv("12.30 Beta1 순서별 충전소_군구.csv", encoding = "CP949", index = False)